In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as op

In [0]:
!pip install ptan

  Created wheel for ptan: filename=ptan-0.6-cp36-none-any.whl size=23502 sha256=a2f91a15e264be02ea08f472c5b2278af6d5b484c9d6cfbb15b037bbd2b1d4ad
  Stored in directory: /root/.cache/pip/wheels/f0/4b/2f/9a45fd39b0a614a2716bc6128a7f1adb4647f323a2d90783f2
Successfully built ptan
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [0]:
import ptan
import gym

In [0]:
env=gym.make("CartPole-v0")

In [0]:
class net(nn.Module):
  def __init__(self,obsdim,nclasses):
    super().__init__()
    self.fc1=nn.Linear(obsdim,128)
    self.fc2=nn.Linear(128,64)
    self.fc3=nn.Linear(64,nclasses)
  
  def forward(self,x):
    out=f.relu(self.fc1(x))
    out=f.relu(self.fc2(out))
    return self.fc3(out)

In [0]:
obsdim=env.observation_space.shape[0]
nactions=env.action_space.n

In [0]:
model=net(obsdim,nactions)

In [0]:
agent=ptan.agent.PolicyAgent(model,preprocessor=ptan.agent.float32_preprocessor,apply_softmax=True)

In [0]:
gamma=0.99


In [0]:
expsource=ptan.experience.ExperienceSourceFirstLast(env,agent,gamma,steps_count=10)

In [0]:
optimizer=op.Adam(model.parameters(),lr=0.0001)

In [0]:
model.train()

net(
  (fc1): Linear(in_features=4, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [0]:
adsfsfdafsdfs

NameError: ignored

NameError: ignored

In [0]:
# def calcqvals(rewards):
#   x=[]
#   sum1=0
#   for i in reversed(rewards):
#     sum1=gamma*sum1+i
#     x.append(sum1) 
#   return list(reversed(x))

In [0]:
batchepisodes=0
batchst=[]
batchac=[]
batchqvals=[]
batchrew=[]
episodestotrain=4
doneepisodes=0
totalrewards=[]
entropybeta=0.01
kldiver=[]
gmax=[]
gmean=[]
stepidx=0
sumre=0

for e in expsource:
  batchst.append(e.state)
  batchac.append(e.action)

  sumre+=e.reward
  meanre=sumre/(stepidx+1)
  batchrew.append(meanre)

  if e.last_state is None:
    batchepisodes+=1
    batchqvals.extend(batchrew)
    batchrew.clear()
  
  newreward=expsource.pop_total_rewards()

  if newreward:
    doneepisodes+=1
    totalrewards.append(newreward[0])
    meanreward=np.mean(totalrewards[-100:])
    print("mean reward:-",meanreward)
    if meanreward>195:
      print("Solved")
      break

  if batchepisodes<episodestotrain:
    continue
  
  
  statesv=torch.tensor(batchst).float()
  statesa=torch.LongTensor(batchac)
  statesqval=torch.tensor(batchqvals).float()

  logits=model(statesv)
  prob=f.softmax(logits,dim=1)
  logprob=f.log_softmax(logits,dim=1)
  logitsprob=-(statesqval*logprob[range(len(statesv)),statesa])
  # print(logitsprob.shape)
  # print(statesv.shape)
  # print(statesa.shape)
  # print(statesqval.shape)
  entropy=(-(prob*logprob).sum(dim=1)).mean()
  losspolicyv=logitsprob.mean()
  entropylossv=-entropybeta*entropy
  lossv=entropylossv+losspolicyv

  optimizer.zero_grad()
  lossv.backward()
  optimizer.step()

  newstatesprob=f.softmax(model(statesv.float()),dim=1)
  kldiv= (((prob/newstatesprob).log())*prob).sum(dim=1).mean()
  kldiver.append(kldiv)

  gradmax=0
  gradmean=0
  gradcount=0
  for p in model.parameters():
    gradmax=max(gradmax,p.grad.abs().max().item())
    gradmeans=(p.grad**2).mean().sqrt().item()
    gradcount+=1
  
  gmax.append(gradmax)
  gmean.append(gradmeans)

  batchst.clear()
  batchrew.clear()
  batchac.clear()

Streaming output truncated to the last 5000 lines.
mean reward:- 24.24
mean reward:- 24.26
mean reward:- 24.38
mean reward:- 24.39
mean reward:- 24.54
mean reward:- 24.61
mean reward:- 24.72
mean reward:- 24.7
mean reward:- 24.7
mean reward:- 24.89
mean reward:- 24.87
mean reward:- 24.8
mean reward:- 24.82
mean reward:- 24.87
mean reward:- 24.87
mean reward:- 24.92
mean reward:- 24.9
mean reward:- 24.89
mean reward:- 24.9
mean reward:- 24.88
mean reward:- 24.9
mean reward:- 24.86
mean reward:- 24.93
mean reward:- 24.91
mean reward:- 24.83
mean reward:- 24.86
mean reward:- 24.98
mean reward:- 24.93
mean reward:- 25.01
mean reward:- 25.09
mean reward:- 25.09
mean reward:- 25.06
mean reward:- 25.09
mean reward:- 25.13
mean reward:- 25.09
mean reward:- 25.13
mean reward:- 25.12
mean reward:- 25.28
mean reward:- 25.2
mean reward:- 25.29
mean reward:- 25.3
mean reward:- 25.36
mean reward:- 25.45
mean reward:- 25.52
mean reward:- 25.53
mean reward:- 25.61
mean reward:- 25.69
mean reward:- 25.

KeyboardInterrupt: ignored